# Übungsblatt 2

Lernziele:
- Klassifikation mit Bayes-Klassifikatoren



In [1]:
import pandas as pd
import numpy as np

Schritt 1: Lesen Sie die Trainings-Daten aus 'data/2d_samples_4cl_train.csv' ein!

In [2]:
df_train = pd.read_csv("data/2d_samples_4cl_train.csv")
df_train

,x1,x2,Label
0,25.5428,8.12002,A
1,28.7852,10.51880,A
2,31.1303,9.20584,B
3,31.1073,9.47408,B
4,31.2040,9.33927,B
...,...,...,...
1995,29.6667,11.06620,A
1996,27.5088,10.48770,A
1997,26.6341,5.35673,D
1998,30.6132,4.39621,C


Schritt 2: Berechnen Sie die Mittelwertvektoren und Kovarianzmatrizen für jede der vier Klassen A,B,C,D sowie deren a-Priori-Wahrscheinlichkeiten.

In [3]:
means = df_train.groupby("Label").mean()
means

,x1,x2
Label,,
A,28.003468,9.994049
B,31.930544,9.938789
C,29.982265,4.974310
D,25.984134,4.940551


In [4]:
covariance = df_train.groupby("Label").cov()
covariance

x1        x2
Label                       
A     x1  2.751974  1.770742
      x2  1.770742  1.277284
B     x1  2.597023  1.216857
      x2  1.216857  1.214641
C     x1  0.850491 -0.685022
      x2 -0.685022  0.893958
D     x1  0.680394 -0.006168
      x2 -0.006168  0.760531

In [5]:
probabilities = df_train.groupby("Label").size().div(len(df_train))
probabilities

Label
A    0.500
B    0.250
C    0.125
D    0.125
dtype: float64

Schritt 3: Definieren Sie eine Funktion norm_pdf(x,mu,sigma), die für einen Vektor _x_ und einen Mittelwertvektor _mu_ und eine Kovarianzmatrix _sigma_ die Normalverteilungs-Wahrscheinlichkeit berechnet. 

In [6]:
def norm_pdf(x, mu, sigma):
    return np.linalg.det( (2 * np.pi * sigma) )**(-0.5) * np.exp( -0.5 * (x - mu) @ np.linalg.inv(sigma) @ (x - mu).T ) 

In [7]:
vector_class_a = df_train[df_train["Label"] == "A"][["x1", "x2"]].iloc[0]
means_class_a = means.loc["A"].values
covariance_matrix_class_a = covariance.loc["A"].values

vector_class_a, means_class_a, covariance_matrix_class_a

(x1    25.54280
 x2     8.12002
 Name: 0, dtype: float64,
 array([28.0034677 ,  9.99404851]),
 array([[2.75197442, 1.77074197],
        [1.77074197, 1.27728384]]))

In [8]:
norm_pdf(vector_class_a, means_class_a, covariance_matrix_class_a)

0.06329202202822354

Schritt 4: Schreiben Sie eine Funktion predict(X), die für die Muster in X eine Klassifikation mit einem Bayes-Klassifikator durchführt.

In [9]:
def predict(X, df):
    classes =  df["Label"].unique()
    mean_vector = df.groupby("Label").mean()
    covs = df.groupby("Label").cov()
    probs = df.groupby("Label").size().div(len(df))
    results = [probs.loc[current_class] * norm_pdf(X, mean_vector.loc[current_class].values, covs.loc[current_class].values) for current_class in classes]
    class_index = np.argmax(results)
    return classes[class_index]

In [10]:
def predict_dataset(X):
    classes =  X["Label"].unique()
    mean_vector = X.groupby("Label").mean()
    covs = X.groupby("Label").cov()
    probs = X.groupby("Label").size().div(len(X))
    predictions = []
    for i in range(len(X)):
        results = [probs.loc[current_class] * 
                       norm_pdf(X[["x1", "x2"]].iloc[i], mean_vector.loc[current_class].values, covs.loc[current_class].values) for current_class in classes]
        predictions.append(classes[np.argmax(results)])
    return predictions

In [11]:
predict(vector_class_a, df_train)

'A'

Schritt 5: Lesen Sie die Test-Daten aus 'data/2d_samples_4cl_test.csv' ein!

In [12]:
df_test = pd.read_csv("data/2d_samples_4cl_test.csv")
df_test

,x1,x2,Label
0,29.2656,4.81426,C
1,28.7640,8.50892,B
2,25.8326,4.19630,D
3,32.1170,9.08867,B
4,28.6760,10.74350,A
...,...,...,...
1995,26.8759,4.59306,D
1996,31.1377,11.84850,A
1997,28.6871,5.94006,C
1998,27.3712,9.81707,A


Schritt 6: Klassifizieren Sie die Testdaten und geben Sie die Anzahl der falsch klassifizierten Muster aus

In [13]:
length = len(df_test)
correct = np.sum([predict(df_test[["x1", "x2"]].iloc[i], df_test) for i in range(length)] == df_test["Label"])
false = length - correct
print(f"Correct: {correct}; False: {false}; Samples: {length}")

Correct: 1977; False: 23; Samples: 2000


In [14]:
dataset_classification = predict_dataset(df_test)
correct_dataset = np.sum(dataset_classification == df_test["Label"])
false_dataset = length - correct_dataset
print(f"Correct: {correct_dataset}; False: {false_dataset}; Samples: {length}")

Correct: 1977; False: 23; Samples: 2000
